In [1]:
import pandas as pd
import numpy as np 
import yfinance as yf
import talib
import datetime as dt
from pandas_datareader import data as pdr
yf.pdr_override()
import pandas_ta as ta

pd.set_option('display.max_rows',None)

In [2]:
import chart_studio.plotly as py
import plotly.offline as plyo
plyo.init_notebook_mode(connected=True)
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
ticker = input("Enter Stock Ticker : ")
print(ticker)

startyear=2012
startmonth=1
startday=1

start=dt.datetime(startyear,startmonth,startday)

end=dt.datetime.now()

df=pdr.get_data_yahoo(ticker,start,end)

Enter Stock Ticker :  BHEL.NS


BHEL.NS
[*********************100%***********************]  1 of 1 completed


In [4]:
data = pd.DataFrame({'Close':df['Close'],
                     'Force_Index(2-period)':df['Close'].diff(2)*df['Volume'],
                     'Force_Index(10-period)':talib.EMA(df['Close'].diff(1)*df['Volume'],timeperiod = 10)})

In [5]:
data = data.dropna(axis=0)
data = data.reset_index()

In [6]:
for i in data.index:
    if (((data['Force_Index(2-period)'][i]>0)&(data['Force_Index(10-period)'][i]>0))&((data['Force_Index(2-period)'].shift(1)[i]<0)|(data['Force_Index(10-period)'].shift(1)[i]<0))):
        print("Buy Call at " + str(data['Close'][i]) + " on " + str(data['Date'][i]))
    if (((data['Force_Index(2-period)'][i]<0)&(data['Force_Index(10-period)'][i]<0))&((data['Force_Index(2-period)'].shift(1)[i]>0)|(data['Force_Index(10-period)'].shift(1)[i]>0))):
        print("Sell Call at " + str(data['Close'][i]) + " on " + str(data['Date'][i]))
    else:
        pass

Sell Call at 176.8333282470703 on 2012-01-19 00:00:00
Buy Call at 182.5 on 2012-01-20 00:00:00
Sell Call at 182.39999389648438 on 2012-01-27 00:00:00
Sell Call at 173.39999389648438 on 2012-02-07 00:00:00
Sell Call at 173.73333740234375 on 2012-02-10 00:00:00
Buy Call at 184.73333740234375 on 2012-02-15 00:00:00
Buy Call at 205.6666717529297 on 2012-02-28 00:00:00
Sell Call at 183.73333740234375 on 2012-03-06 00:00:00
Buy Call at 189.96665954589844 on 2012-03-12 00:00:00
Sell Call at 189.06666564941406 on 2012-03-15 00:00:00
Sell Call at 174.1666717529297 on 2012-03-22 00:00:00
Buy Call at 176.06666564941406 on 2012-04-03 00:00:00
Sell Call at 167.06666564941406 on 2012-04-11 00:00:00
Buy Call at 171.60000610351562 on 2012-04-12 00:00:00
Sell Call at 167.03334045410156 on 2012-04-19 00:00:00
Sell Call at 142.6999969482422 on 2012-05-04 00:00:00
Buy Call at 151.3000030517578 on 2012-05-07 00:00:00
Sell Call at 144.06666564941406 on 2012-05-09 00:00:00
Sell Call at 144.76666259765625 on 

In [7]:

pos = 0

positions_list = []

buy_calls = []
sell_calls = []
buy_prices = []
sell_prices = []
buy_dates = []
sell_dates = []


for i in data.index:
    if (((data['Force_Index(2-period)'][i]>0)&(data['Force_Index(10-period)'][i]>0))&((data['Force_Index(2-period)'].shift(1)[i]<0)|(data['Force_Index(10-period)'].shift(1)[i]<0))):
        if (pos == 0):
            bp = data['Close'][i] 
            pos = 1
            #print('Buy Call at '+str(bp)+' on '+ str(data['Date'][i]))   
            positions_list.append(pos)
            
            call_type = "buy"
            buy_date = data['Date'][i]
            
            buy_calls.append(call_type)
            buy_prices.append(bp)
            buy_dates.append(buy_date)
            
       # if (pos==1):
       #     pos = 1  

    """    
        if (pos == 1) & ((data['Close'][i]/bp)>1.1):
            pos = 0
            call_type = "sell"
            sp = data['Close'][i] 
            sell_date = data['Date'][i]
            
            sell_calls.append(call_type)
            sell_prices.append(sp)
            sell_dates.append(sell_date) 
    """        
            
    if (((data['Force_Index(2-period)'][i]<0)&(data['Force_Index(10-period)'][i]<0))&((data['Force_Index(2-period)'].shift(1)[i]>0)|(data['Force_Index(10-period)'].shift(1)[i]>0))):
        if (pos == 1):
            sp = data['Close'][i] 
            pos = 0
            #print('Sell Call at '+str(sp)+' on '+ str(data['Date'][i]))
            positions_list.append(pos)
            
            call_type = "sell"
            sp = data['Close'][i] 
            sell_date = data['Date'][i]
            
            sell_calls.append(call_type)
            sell_prices.append(sp)
            sell_dates.append(sell_date)
            

    # Code for Take Profit (10%)    
            
    if (len(positions_list)>0):
        
        if(positions_list[-1] == 1) & (len(buy_prices)>=1):
            
            if (data['Close'][i] >= (buy_prices[-1] * 1.1)):
                sp = data['Close'][i]
                pos = 0
                positions_list.append(pos)
                
                call_type = 'sell'
                sp = data['Close'][i] 
                sell_date = data['Date'][i]
            
                sell_calls.append(call_type)
                sell_prices.append(sp)
                sell_dates.append(sell_date)
            
            
    # Code for Stoploss (5%)    
            
    if (len(positions_list)>0): 
        
        if(positions_list[-1] == 1) & (len(buy_prices)>=1):
            if (data['Close'][i] <= (buy_prices[-1] * 0.95)):
                sp = data['Close'][i]
                pos = 0
                positions_list.append(pos)

                call_type = 'sell'
                sp = data['Close'][i] 
                sell_date = data['Date'][i]
            
                sell_calls.append(call_type)
                sell_prices.append(sp)
                sell_dates.append(sell_date)            
            
            
            
            
        #if (pos == 0):
        #    pos = 0 
    


    else:
        pass

In [8]:
buy_all = pd.DataFrame({'Calls':buy_calls,
                        'Date':buy_dates,
                        'Price':buy_prices,
                        })

sell_all = pd.DataFrame({'Calls':sell_calls,
                        'Date':sell_dates,
                        'Price':sell_prices,
                        })

### Accounting for commissions

1% for Buy Orders

2% for Sell Orders

In [9]:
buy_all['Price'] = buy_all['Price']*1.01
sell_all['Price'] = sell_all['Price']*0.98

In [10]:
all_calls = pd.concat(objs=[buy_all,sell_all],axis=0).sort_values('Date')

In [11]:
all_calls['P&L(Amount)'] = all_calls.groupby(all_calls.Calls.eq('buy').cumsum())['Price'].diff()
all_calls['P&L(%)'] = (all_calls.groupby(all_calls.Calls.eq('buy').cumsum())['Price'].pct_change())*100
all_calls['Time in Trade'] = all_calls.groupby(all_calls.Calls.eq('buy').cumsum())['Date'].diff()

In [12]:
print("Strategy Success Rate : {:.2f}%".format(((all_calls.loc[all_calls['P&L(%)']>=0]['P&L(%)'].count())/(all_calls['P&L(%)'].count()))*100))

Strategy Success Rate : 21.20%


In [13]:
print("Average Gain of all profitable trades : {:.2f}%".format(all_calls.loc[all_calls['P&L(%)']>=0]['P&L(%)'].mean()))

Average Gain of all profitable trades : 8.86%


In [14]:
print("Maximum Gain of all profitable trades : {:.2f}%".format(all_calls.loc[all_calls['P&L(%)']>=0]['P&L(%)'].max()))

Maximum Gain of all profitable trades : 18.24%


In [15]:
print("Average Loss of all Loss making trades : {:.2f}%".format(all_calls.loc[all_calls['P&L(%)']<0]['P&L(%)'].mean()))

Average Loss of all Loss making trades : -6.04%


In [16]:
print("Maximum Loss of all Loss making trades : {:.2f}%".format(all_calls.loc[all_calls['P&L(%)']<0]['P&L(%)'].min()))

Maximum Loss of all Loss making trades : -14.29%


In [17]:
print("Average Time in a Trade : {:.0f} days".format(all_calls['Time in Trade'].dt.days.mean()))

Average Time in a Trade : 8 days


In [18]:
def P_and_L_calc(all_calls_data):
    
    all_calls_data['P&L(Amount)'] = all_calls_data.groupby(all_calls_data.Calls.eq('buy').cumsum())['Price'].diff()
    all_calls_data['P&L(%)'] = (all_calls_data.groupby(all_calls_data.Calls.eq('buy').cumsum())['Price'].pct_change())*100
    all_calls_data['Time in Trade'] = all_calls_data.groupby(all_calls_data.Calls.eq('buy').cumsum())['Date'].diff()
    
    P_n_L = pd.DataFrame({'Strategy Success Rate (%)':round(all_calls_data.loc[all_calls_data['P&L(%)']>=0]['P&L(%)'].count()/(all_calls_data['P&L(%)'].count())*100,2),
                         'Avg. Gain of Profitable Trades (%)':round(all_calls_data.loc[all_calls_data['P&L(%)']>=0]['P&L(%)'].mean(),2),
                         'Max. Gain of Profitable Trades (%)':round(all_calls_data.loc[all_calls_data['P&L(%)']>=0]['P&L(%)'].max(),2),
                         'Avg. Loss of all loss making Trades (%)':round(all_calls_data.loc[all_calls_data['P&L(%)']<0]['P&L(%)'].mean(),2),
                         'Max. Loss of all loss making Trades (%)':round(all_calls_data.loc[all_calls_data['P&L(%)']<0]['P&L(%)'].min(),2),
                         'Avg. Time in a Trade (days)':round(all_calls_data['Time in Trade'].dt.days.mean(),0)
                         },index=[0])
    
    return P_n_L

In [19]:
P_and_L_calc(all_calls)

,Strategy Success Rate (%),Avg. Gain of Profitable Trades (%),Max. Gain of Profitable Trades (%),Avg. Loss of all loss making Trades (%),Max. Loss of all loss making Trades (%),Avg. Time in a Trade (days)
0,21.2,8.86,18.24,-6.04,-14.29,8.0


In [20]:
all_calls.tail()

,Calls,Date,Price,P&L(Amount),P&L(%),Time in Trade
215,buy,2022-09-20,60.700998,NaN,NaN,NaT
215,sell,2022-09-21,57.917999,-2.783000,-4.584768,1 days
216,buy,2022-09-29,59.085000,NaN,NaN,NaT
216,sell,2022-10-18,66.835997,7.750997,13.118384,19 days
217,buy,2022-10-25,71.810998,NaN,NaN,NaT
